In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

In [ ]:
from PyPDF2 import PdfReader
from langchain_community.vectorstores import Cassandra
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

from datasets import load_dataset

import cassio

In [ ]:
from dotenv import load_dotenv
load_dotenv()
#import ur variables here

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('budget_speech.pdf')

In [ ]:
from typing_extensions import Concantenate
raw_text=''
for i ,page in enumerate(pdfreader.pages):
content=page.extract_text()
if content:
  rawtext+=content

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
llm=ChatGroq(model_name='mixtral-8x7b-32768',groq_api_key=groq_api_key)
embeddings=HuggingFaceEmbeddings(model='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
import cassio # Ensure cassio is imported here if not globally available in this scope
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name='qa_mini_demo',
    session=cassio.get_session(),
    keyspace=cassio.get_keyspace()
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))

In [ ]:
first_question=True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()

    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()
    
    if query_text.lower() == "quit":
        break
    if query_text == "": 
        continue
    first_question = False

print(f"\nQUESTION: {query_text}") 
answer = astra_vector_index.query(query_text, llm=llm).strip()
print(answer)

print("FIRST DOCUMENTS BY RELEVANCE")
for doc,score in astra_vector_store.similarity_serach_with_score(query_text,k=4):
    print(score,doc.page_content)
    print(f'    [{score:.4f}] "{doc.page_content[:84]} ..."')


Nice, this is a **very real-world AI app loop** you’re looking at. Let’s slow it down and demystify it completely.

First: **yes, this is Python** 🐍
Nothing exotic. Just a REPL-style loop around a **vector database + LLM**.

---

## 1️⃣ What is this code doing (big picture)

> This code creates an **interactive Q&A chatbot** that:
>
> * Takes a question from the user
> * Uses a **vector database (Astra DB)** to retrieve relevant documents
> * Uses an **LLM** to generate an answer
> * Shows which documents were used (with similarity scores)
> * Repeats until the user types `quit`

So this is **RAG (Retrieval-Augmented Generation)** in action.

---

## 2️⃣ Which language is this?

### ✅ Python

Clues:

* `while True`
* `input()`
* `.strip()`
* `%` string formatting
* `.lower()`

This is **plain Python**, not LangChain magic.

---

## 3️⃣ Line-by-line explanation

---

### 🔹 Control variable

```python
first_question = True
```

Used to slightly change the prompt text for UX.

---

### 🔹 Infinite loop (chat loop)

```python
while True:
```

This means:

> Keep asking questions **until user exits**

---

### 🔹 User input (first vs next question)

```python
if first_question:
    query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
else:
    query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()
```

Why this exists:

* First question → different wording
* After that → “next question”

`.strip()` removes:

* leading spaces
* trailing spaces
* accidental newline

---

### 🔹 Exit condition

```python
if query_text.lower() == "quit":
    break
```

* `.lower()` → case-insensitive
* `break` → exit `while True`

---

### 🔹 Ignore empty input

```python
if query_text == "":
    continue
```

If user presses Enter without typing:

* `continue` → skip rest of loop
* Ask again

---

### 🔹 After first iteration

```python
first_question = False
```

Now the next prompt will say:

> “What’s your next question…”

---

## 4️⃣ Printing the question

```python
print("\nQUESTION: \"%s\"" % query_text)
```

Old-school Python formatting:

```python
"%s" % variable
```

Same as:

```python
f"{query_text}"
```

---

## 5️⃣ 🔥 The MOST IMPORTANT LINE

```python
answer = astra_vector_index.query(query_text, llm=llm).strip()
```

This is where the magic happens.

### What is happening here?

1. `query_text` → converted to embedding
2. Astra DB vector index:

   * Finds most relevant documents
3. Those documents are sent to the LLM
4. LLM generates an answer
5. `.strip()` cleans whitespace

This is **RAG in one line**.

---

### What is `astra_vector_index`?

Likely:

* A wrapper around **Astra DB vector search**
* Abstracts:

  * embedding
  * retrieval
  * LLM call

Think:

```text
User Question
   ↓
Vector Search
   ↓
Relevant Docs
   ↓
LLM
   ↓
Answer
```

---

## 6️⃣ Print the answer

```python
print("ANSWER: \"%s\"\n" % answer)
```

Just output to console.

---

## 7️⃣ Show retrieved documents (transparency 👌)

```python
print("FIRST DOCUMENTS BY RELEVANCE:")
```

This is **debug / explainability**.

---

### 🔹 Similarity search

```python
for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
```

Meaning:

* Convert query → embedding
* Find top `k=4` most similar docs
* Return:

  * `doc` → document object
  * `score` → similarity score

---

### 🔹 Print relevance + snippet

```python
print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))
```

What this prints:

* Similarity score (4 decimal places)
* First 84 characters of document

Example:

```
[0.9123] "Cassandra is a distributed NoSQL database ..."
```

This is **excellent practice** — lets you see *why* the answer was generated.

---

## 8️⃣ Objects you didn’t see defined (but should know)

### `astra_vector_store`

* Vector DB interface
* Stores embeddings + documents

### `astra_vector_index`

* Higher-level abstraction
* Probably wraps:

  * retriever
  * prompt
  * LLM call

### `llm`

* ChatGPT / Groq / OpenAI / etc.

---

## 9️⃣ What kind of app is this?

This is a:

* CLI chatbot
* Retrieval-Augmented QA system
* Knowledge-base assistant

**Exactly the kind of thing companies build.**

---

## 🔥 TL;DR (lock this in)

* Language → **Python**
* Pattern → **RAG**
* Loop → CLI chat interface
* Vector DB → Astra DB
* LLM → generates final answer
* Docs printed → transparency & debugging

---

## 🚀 Why this code is GOOD

✔ Clean loop
✔ Clear exit condition
✔ Shows retrieved documents
✔ Not hiding magic

This is **production-quality learning code**.

---

If you want next, I can:

* Rewrite this in **LangChain LCEL**
* Show how Astra DB stores embeddings
* Convert this to **Streamlit / Gradio**
* Explain similarity scores mathematically
* Compare this to OpenAI Assistants API

Just say 👊
